<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/kimjisan/0917_microev_pharm_anti%26medication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [5]:
microev=pd.read_csv('/content/drive/MyDrive/mimiciv_unzip/hosp/microbiologyevents.csv')
pharm=pd.read_csv('/content/drive/MyDrive/mimiciv_unzip/hosp/pharmacy.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16,18,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
pd.set_option('display.max_columns',None)

In [23]:
print(microev[['micro_specimen_id','org_name','comments','']].head())

    microevent_id  subject_id  hadm_id  micro_specimen_id  \
19             20    10000032      NaN            5842819   
20             21    10000032      NaN            5842819   
21             22    10000032      NaN            5842819   
22             23    10000032      NaN            5842819   
40             41    10000108      NaN            3019793   

              chartdate            charttime  spec_itemid spec_type_desc  \
19  2180-06-26 00:00:00  2180-06-26 18:30:00        70079          URINE   
20  2180-06-26 00:00:00  2180-06-26 18:30:00        70079          URINE   
21  2180-06-26 00:00:00  2180-06-26 18:30:00        70079          URINE   
22  2180-06-26 00:00:00  2180-06-26 18:30:00        70079          URINE   
40  2163-09-27 00:00:00  2163-09-27 22:27:00        70067           SWAB   

    test_seq            storedate            storetime  test_itemid  \
19         1  2180-06-29 00:00:00  2180-06-29 14:32:00        90039   
20         1  2180-06-29 00:00:00 

In [6]:
microev=microev[microev['org_name'].notnull()]
microev_bld=microev[(microev['spec_type_desc']=='BLOOD CULTURE') & (microev['org_name']!='CANCELLED')]

In [7]:
subject_id_bld=microev_bld['subject_id'].unique()
len(subject_id_bld)

11556

혈액배양을 수행한 환자의 pharmacy 기록 추출

In [8]:
pharm_bld=pharm[pharm['subject_id'].isin(subject_id_bld)]
len(pharm_bld['subject_id'].unique())

11488

In [9]:
pd.set_option('display.max_rows', 20)

In [10]:
pharm_bld=pd.DataFrame(pharm_bld)

In [11]:
pharm_bld['medication']=pharm_bld['medication'].astype(str)
pharm_bld['medication']=pharm_bld['medication'].map(str.upper)

In [12]:
pharm_bld['medication'].value_counts()

NAN                            248399
INSULIN                        205460
SODIUM CHLORIDE 0.9%  FLUSH    113227
POTASSIUM CHLORIDE              97010
FUROSEMIDE                      92392
                                ...  
ATOVAQUONE                          1
RYTHMOL                             1
VENLAF                              1
NYST                                1
BENAZEPRIL-HCTZ 20-25MG             1
Name: medication, Length: 4068, dtype: int64

In [13]:
len(pharm_bld['medication'])

3413252

In [14]:
microev_bld['ab_name']=microev_bld['ab_name'].replace('TRIMETHOPRIM/SULFA','TRIMETHOPRIM')
microev_bld['ab_name']=microev_bld['ab_name'].replace('PIPERACILLIN/TAZO','PIPERACILLIN')
ab_list = microev_bld['ab_name'].unique()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
print(ab_list) #'TRIMETHOPRIM/SULFA', 'PIPERACILLIN/TAZO'
ab_list=ab_list.astype(str)

['TRIMETHOPRIM' 'GENTAMICIN' 'TOBRAMYCIN' 'CEFTAZIDIME' 'CEFTRIAXONE'
 'CIPROFLOXACIN' 'PIPERACILLIN' 'CEFEPIME' 'MEROPENEM' nan 'PENICILLIN G'
 'AMPICILLIN' 'ERYTHROMYCIN' 'CLINDAMYCIN' 'VANCOMYCIN' 'CEFAZOLIN'
 'AMPICILLIN/SULBACTAM' 'TETRACYCLINE' 'OXACILLIN' 'LEVOFLOXACIN'
 'RIFAMPIN' 'DAPTOMYCIN' 'LINEZOLID' 'AMIKACIN' 'IMIPENEM' 'CEFUROXIME'
 'NITROFURANTOIN']


ab_list에 있는 문자열을 포함하는 문자열을 가진 행만 추출

In [16]:
# join함수를 이용하여 이어주고 contains 함수에 넣기
ab_list_patt = '|'.join(ab_list)
pharm_bld_ab = pharm_bld[pharm_bld['medication'].str.contains(ab_list_patt)]

In [17]:
pharm_bld_ab['medication'].unique()

array(['NITROFURANTOIN MONOHYD (MACROBID)', 'CEFAZOLIN', 'AMPICILLIN',
       'CEFTRIAXONE', 'SULFAMETH/TRIMETHOPRIM DS', 'CIPROFLOXACIN',
       'CIPROFLOXACIN HCL', 'CEFTAZIDIME', 'VANCOMYCIN',
       'PIPERACILLIN-TAZOBACTAM', 'VANCOMYCIN ORAL LIQUID', 'CEFEPIME',
       'MEROPENEM', 'LEVOFLOXACIN', 'VANCOMYCIN ENEMA',
       'AMPICILLIN-SULBACTAM', 'TOBRAMYCIN', 'LINEZOLID', 'DAPTOMYCIN',
       'DAPTOMYCIN-HEPARIN LOCK', 'GENTAMICIN', 'RIFAMPIN', 'CLINDAMYCIN',
       'AMIKACIN', 'CEFEPIME DESENSITIZATION',
       'VANCOMYCIN DESENSITIZATION', 'SULFAMETH/TRIMETHOPRIM SS',
       'ERYTHROMYCIN 0.5% OPHTH OINT', 'SULFAMETH/TRIMETHOPRIM',
       'SULFAMETH/TRIMETHOPRIM SUSPENSION', 'DICLOXACILLIN',
       'PIPERACILLIN-TAZOBACTAM NA', 'SULFAMETHOXAZOLE-TRIMETHOPRIM',
       'PENICILLIN G POTASSIUM', 'TETRACYCLINE HCL', 'ERYTHROMYCIN',
       'IMIPENEM-CILASTATIN', 'NITROFURANTOIN (MACRODANTIN)',
       'ERYTHROMYCIN ETHYLSUCCINATE SUSPENSION',
       'TOBRAMYCIN-DEXAMETHASONE OPHTH S

In [18]:
len(pharm_bld_ab['medication'].unique())

123

In [19]:
pharm_bld_ab['medication'].value_counts().head(20)

VANCOMYCIN                    70416
CEFEPIME                      20410
CEFTRIAXONE                   15133
PIPERACILLIN-TAZOBACTAM       14759
CIPROFLOXACIN HCL             11959
MEROPENEM                     10912
LEVOFLOXACIN                   8407
CIPROFLOXACIN                  6777
CEFAZOLIN                      6660
VANCOMYCIN ORAL LIQUID         5501
SULFAMETH/TRIMETHOPRIM SS      5105
DAPTOMYCIN                     4543
SULFAMETH/TRIMETHOPRIM DS      4164
CEFTAZIDIME                    3721
LINEZOLID                      3677
AMPICILLIN-SULBACTAM           3279
CLINDAMYCIN                    2299
GENTAMICIN                     2021
PIPERACILLIN-TAZOBACTAM NA     1972
AMPICILLIN                     1589
Name: medication, dtype: int64

In [ ]:
pharm_bld_ab.head(10)

### micro event의 항생제감수성 결과와 실제로 처방된 medication과 비교해서 label을 만들 수 있을까해서 case를 뽑아 봤음.

In [58]:
submi=microev_bld['subject_id'].unique()
print(len(microev_bld['subject_id'].unique()))

11556


###microevent에서 blood culture를 받은 subject_id는 11556명이다

In [71]:
# blood culture 받은 랜덤 환자의 항생제 감수성과 처방된 항생제를 print
def ant_medi():
  x=np.random.randint(0,11555)
  t=submi[x]
  ph_ex=pharm_bld_ab[pharm_bld_ab['subject_id']==t]
  mi_ex=microev_bld[microev_bld['subject_id']==t]
  print(ph_ex[['subject_id','medication']])
  print(mi_ex[['subject_id','org_name','ab_name','interpretation']])

In [67]:
ant_medi()

         subject_id               medication
3756978    12627028              CEFTRIAXONE
3757015    12627028        CIPROFLOXACIN HCL
3757017    12627028        CIPROFLOXACIN HCL
3757026    12627028        CIPROFLOXACIN HCL
3757101    12627028                CEFAZOLIN
3757114    12627028  PIPERACILLIN-TAZOBACTAM
3757115    12627028  PIPERACILLIN-TAZOBACTAM
3757126    12627028        CIPROFLOXACIN HCL
3757156    12627028        CIPROFLOXACIN HCL
        subject_id            org_name               ab_name interpretation
891858    12627028  KLEBSIELLA OXYTOCA             CEFAZOLIN              S
891859    12627028  KLEBSIELLA OXYTOCA          TRIMETHOPRIM              S
891860    12627028  KLEBSIELLA OXYTOCA            GENTAMICIN              S
891861    12627028  KLEBSIELLA OXYTOCA            TOBRAMYCIN              S
891862    12627028  KLEBSIELLA OXYTOCA           CEFTAZIDIME              S
891863    12627028  KLEBSIELLA OXYTOCA           CEFTRIAXONE              S
891864    12627028

In [68]:
ant_medi()

        subject_id                 medication
643189    10465217  SULFAMETH/TRIMETHOPRIM SS
643199    10465217  SULFAMETH/TRIMETHOPRIM SS
643241    10465217  SULFAMETH/TRIMETHOPRIM SS
643257    10465217  SULFAMETH/TRIMETHOPRIM SS
        subject_id                            org_name ab_name interpretation
156614    10465217  STAPHYLOCOCCUS, COAGULASE NEGATIVE     NaN            NaN
156615    10465217  STAPHYLOCOCCUS, COAGULASE NEGATIVE     NaN            NaN


In [69]:
ant_medi()

         subject_id                  medication
9827068    16879557                  VANCOMYCIN
9827088    16879557                   MEROPENEM
9827104    16879557                   MEROPENEM
9827118    16879557                  VANCOMYCIN
9827130    16879557                  VANCOMYCIN
9827141    16879557                   MEROPENEM
9827142    16879557                  VANCOMYCIN
9827150    16879557               CIPROFLOXACIN
9827164    16879557  PIPERACILLIN-TAZOBACTAM NA
9827203    16879557                LEVOFLOXACIN
9827214    16879557                   CEFAZOLIN
         subject_id          org_name ab_name interpretation
2332228    16879557  CANDIDA ALBICANS     NaN            NaN


In [70]:
ant_medi()

          subject_id         medication
10165166    17111670  CIPROFLOXACIN HCL
10165169    17111670      CIPROFLOXACIN
10165195    17111670         VANCOMYCIN
10165220    17111670         GENTAMICIN
10165223    17111670         VANCOMYCIN
...              ...                ...
10165448    17111670  CIPROFLOXACIN HCL
10165450    17111670  CIPROFLOXACIN HCL
10165463    17111670         VANCOMYCIN
10165470    17111670  CIPROFLOXACIN HCL
10165471    17111670           CEFEPIME

[22 rows x 2 columns]
         subject_id                      org_name               ab_name  \
2413590    17111670         KLEBSIELLA PNEUMONIAE             CEFAZOLIN   
2413591    17111670         KLEBSIELLA PNEUMONIAE          TRIMETHOPRIM   
2413592    17111670         KLEBSIELLA PNEUMONIAE            GENTAMICIN   
2413593    17111670         KLEBSIELLA PNEUMONIAE            TOBRAMYCIN   
2413594    17111670         KLEBSIELLA PNEUMONIAE              AMIKACIN   
2413595    17111670         KLEBSIELLA PNEUMONI

In [77]:
ant_medi()

         subject_id   medication
9146516    16389432     CEFEPIME
9146518    16389432   VANCOMYCIN
9146520    16389432  CEFTAZIDIME
9146525    16389432   VANCOMYCIN
9146528    16389432  CEFTAZIDIME
9146565    16389432   VANCOMYCIN
9146571    16389432   VANCOMYCIN
9146621    16389432  CEFTAZIDIME
         subject_id                            org_name ab_name interpretation
2173774    16389432  STAPHYLOCOCCUS, COAGULASE NEGATIVE     NaN            NaN


In [80]:
ant_medi()

         subject_id    medication
9077005    16336326  LEVOFLOXACIN
9077094    16336326      CEFEPIME
9077100    16336326    VANCOMYCIN
9077107    16336326    VANCOMYCIN
9077112    16336326    VANCOMYCIN
9077115    16336326      CEFEPIME
9077125    16336326    VANCOMYCIN
9077129    16336326    VANCOMYCIN
         subject_id             org_name       ab_name interpretation
2157511    16336326  STAPH AUREUS COAG +  ERYTHROMYCIN              R
2157512    16336326  STAPH AUREUS COAG +   CLINDAMYCIN              R
2157513    16336326  STAPH AUREUS COAG +  TRIMETHOPRIM              S
2157514    16336326  STAPH AUREUS COAG +    GENTAMICIN              S
2157515    16336326  STAPH AUREUS COAG +    VANCOMYCIN              S
2157516    16336326  STAPH AUREUS COAG +     OXACILLIN              R
2157517    16336326  STAPH AUREUS COAG +  LEVOFLOXACIN              R
2157518    16336326  STAPH AUREUS COAG +      RIFAMPIN              S
2157519    16336326  STAPH AUREUS COAG +           NaN          

In [81]:
ant_medi()

        subject_id                 medication
710991    10514578                CEFTRIAXONE
710995    10514578                  CEFAZOLIN
711005    10514578          CIPROFLOXACIN HCL
711020    10514578                 VANCOMYCIN
711049    10514578                 VANCOMYCIN
711114    10514578  SULFAMETH/TRIMETHOPRIM DS
        subject_id                            org_name       ab_name  \
171235    10514578          STAPHYLOCOCCUS EPIDERMIDIS  ERYTHROMYCIN   
171236    10514578          STAPHYLOCOCCUS EPIDERMIDIS   CLINDAMYCIN   
171237    10514578          STAPHYLOCOCCUS EPIDERMIDIS    GENTAMICIN   
171238    10514578          STAPHYLOCOCCUS EPIDERMIDIS    VANCOMYCIN   
171239    10514578          STAPHYLOCOCCUS EPIDERMIDIS     OXACILLIN   
171240    10514578          STAPHYLOCOCCUS EPIDERMIDIS  LEVOFLOXACIN   
171243    10514578          STAPHYLOCOCCUS EPIDERMIDIS  ERYTHROMYCIN   
171244    10514578          STAPHYLOCOCCUS EPIDERMIDIS   CLINDAMYCIN   
171245    10514578          ST

###medication과 항생제감수성 결과로는 새로운 feature나 label을 확보하기 어려워 보임

- 처방 medication과 감수성검사 약제가 차이가 있는 점
- 감수성검사가 R이 나오더라도 그 항생제에 관해서 잘못되었다 라벨링 하는게 맞는가 > 일반화가 힘들 것 같음

>>차라리 org_name으로 무언가 시도해보는게 나은가 싶었음.
(vital, lab data, 1차 감염부위 등으로 병원균 예측)
병원균 예측 후  guidline에 따라 antibiotic 추천